<a href="https://colab.research.google.com/github/alex-jk/ExData_Plotting1/blob/master/process_studies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This notebook extracts and cleans text from PDF reports for use in training the buyer profile extractor model.**

**Add input and output examples to jsonl file**

In [8]:
#!git clone https://github.com/alex-jk/buyers-ID-manual-project.git
%cd buyers-ID-manual-project
!ls

[Errno 2] No such file or directory: 'buyers-ID-manual-project'
/content/buyers-ID-manual-project
data		     LICENSE		    __pycache__
extraction_utils.py  process_studies.ipynb  README.md


In [9]:
# import json

# input_text = """
# Buyers were willing to travel significant distances to reach underage victims.
# One man cited the Atlanta airport as a regular meeting place.
# """

# output_text = """
# Buyers are often mobile and willing to travel, including to major hubs like the Atlanta airport.
# """

# entry = {"input": input_text.strip(), "output": output_text.strip()}

# with open("data/labeled_chunks.jsonl", "a", encoding="utf-8") as f:
#     f.write(json.dumps(entry) + "\n")

**Import libraries**

In [15]:
# Install specific versions targeting CUDA 12.1 (likely compatible with 12.5 drivers)
!pip install torch==2.3.1+cu121 -f https://download.pytorch.org/whl/torch_stable.html

# Reinstall the other pinned versions
!pip install transformers==4.41.2 accelerate==0.31.0 --no-deps

# Ensure base dependencies are present
!pip install bitsandbytes sentencepiece

print("Installed specific package versions (torch targeting cu121). PLEASE RESTART RUNTIME NOW.")

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.0/781.0 MB ? eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Usi

In [10]:
import pandas as pd
import json
import sys
import os

**Read input output file and write to jsonl**

In [11]:
input_output_df = pd.read_csv("data/buyers_id_manual_input_output.csv")

print(input_output_df.shape)
print(input_output_df.columns)
print("\n-------------------------------\n")
print(input_output_df.head())

(9, 3)
Index(['Study', 'Input', 'Output'], dtype='object')

-------------------------------

                                    Study  \
0  The Shapiro Group Georgia Demand Study   
1  The Shapiro Group Georgia Demand Study   
2  The Shapiro Group Georgia Demand Study   
3  The Shapiro Group Georgia Demand Study   
4  The Shapiro Group Georgia Demand Study   

                                               Input  \
0  Almost half these men are the age 30-39, with ...   
1  The data clearly debunk the myth that CSEC is ...   
2  Not only are 65% of men who buy sex with young...   
3  Craigslist is by far the most efficient medium...   
4  While many of the men who exploit these childr...   

                                              Output  
0  Almost half these men are the age 30-39, with ...  
1  Men who\nrespond to advertisements for sex wit...  
2  65% of men who buy sex with young females do s...  
3  Buyers respond to Craigslist ads 3 times more ...  
4  Nearly half of buyers

In [12]:
input_output_df = input_output_df.drop_duplicates(subset=["Study", "Input", "Output"])

# Convert to JSONL
with open("data/labeled_chunks.jsonl", "w") as f:
    for _, row in input_output_df.iterrows():
        json_obj = {
            "study": row["Study"],
            "input": row["Input"],
            "output": row["Output"]
        }
        f.write(json.dumps(json_obj) + "\n")

**Models for text extraction**

In [13]:
from extraction_utils import create_prompt_messages, extract_information

print("Functions 'create_prompt_messages' and 'extract_information' imported successfully.")

ImportError: libcudnn.so.8: cannot open shared object file: No such file or directory

In [14]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [9]:
# --- Configuration ---
model_id = "microsoft/Phi-3-mini-4k-instruct"

# --- Load Model and Tokenizer ---
# Load the model with 4-bit quantization to save memory (requires bitsandbytes)
# Use device_map="auto" to automatically use GPU if available
try:
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",         # Use GPU if available, otherwise CPU
        torch_dtype="auto",        # Automatically select appropriate dtype
        trust_remote_code=True,    # Phi-3 requires this
        # Optional: uncomment below for 4-bit loading (needs bitsandbytes)
        # load_in_4bit=True,
        # bnb_4bit_compute_dtype=torch.bfloat16
    )
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    print(f"Model '{model_id}' loaded successfully.")

    # --- Create a Hugging Face Pipeline for easier text generation ---
    # Note: max_new_tokens controls how long the generated output can be. Adjust as needed.
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        # Adjust max_new_tokens if your extracted text might be longer
        # Needs to be long enough for the longest expected extraction + "NONE"
        max_new_tokens=256,
        # Temperature=0 means more deterministic output, higher means more creative/random
        # temperature=0.0,
        # top_p=0.95, # Optional: nucleus sampling
        do_sample=False # Set to False for more deterministic extraction
    )

except Exception as e:
    print(f"Error loading model or creating pipeline: {e}")
    print("Ensure you have sufficient RAM/VRAM and necessary libraries installed.")
    print("Consider using Google Colab with a T4 GPU runtime.")
    # Exit if model loading fails
    exit()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


Model 'microsoft/Phi-3-mini-4k-instruct' loaded successfully.


**Example Usage**

In [10]:
input_text_1 = "Almost half these men are the age 30-39, with the next largest group being men under age 30. The mean age is 33 and the median 31. The youngest survey participant was 18, and the oldest was 67."

input_text_2 = "The data clearly debunk the myth that CSEC is a problem relegated to the urban core. Men who respond to advertisements for sex with young females come from all over metro Atlanta, the geographic market where the advertisements in this study were targeted."

input_text_3 = "This paragraph discusses unrelated economic factors in the region and contains no information about buyers or traffickers."

input_text_4 = "Research indicates traffickers often groom potential buyers by displaying luxury goods online and frequenting specific forums known for risky behavior discussions. Victims may appear withdrawn or display signs of coaching. Reporting suspicions anonymously through the national hotline is encouraged."

In [11]:
print("\n--- Extracting Information ---")

# Check if the pipeline object exists before trying to use it
if 'pipe' in locals() and pipe is not None:
    print(f"\nInput 1:\n{input_text_1}")
    extraction_1 = extract_information(input_text_1, pipe)
    print(f"\nExtraction 1:\n{extraction_1}")

    print("-" * 20)

    print(f"\nInput 2:\n{input_text_2}")
    extraction_2 = extract_information(input_text_2, pipe)
    print(f"\nExtraction 2:\n{extraction_2}")

    print("-" * 20)

    print(f"\nInput 3:\n{input_text_3}")
    extraction_3 = extract_information(input_text_3, pipe)
    print(f"\nExtraction 3:\n{extraction_3}")

    print("-" * 20)

    print(f"\nInput 4:\n{input_text_4}")
    extraction_4 = extract_information(input_text_4, pipe)
    print(f"\nExtraction 4:\n{extraction_4}")

else:
    print("\nERROR: The 'pipe' object (text generation pipeline) was not found or not created successfully.")
    print("Please ensure the model loading cell was run successfully after importing functions.")


--- Extracting Information ---

Input 1:
Almost half these men are the age 30-39, with the next largest group being men under age 30. The mean age is 33 and the median 31. The youngest survey participant was 18, and the oldest was 67.


The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


Error during generation: 'DynamicCache' object has no attribute 'get_max_length'

Extraction 1:
Error: Could not generate output. ('DynamicCache' object has no attribute 'get_max_length')
--------------------

Input 2:
The data clearly debunk the myth that CSEC is a problem relegated to the urban core. Men who respond to advertisements for sex with young females come from all over metro Atlanta, the geographic market where the advertisements in this study were targeted.
Error during generation: 'DynamicCache' object has no attribute 'get_max_length'

Extraction 2:
Error: Could not generate output. ('DynamicCache' object has no attribute 'get_max_length')
--------------------

Input 3:
This paragraph discusses unrelated economic factors in the region and contains no information about buyers or traffickers.
Error during generation: 'DynamicCache' object has no attribute 'get_max_length'

Extraction 3:
Error: Could not generate output. ('DynamicCache' object has no attribute 'get_max_leng

In [12]:
# Check installed versions of required packages
!pip list | grep -E 'transformers|torch|accelerate|flash-attn'

accelerate                            1.6.0
sentence-transformers                 3.4.1
torch                                 2.6.0+cu124
torchaudio                            2.6.0+cu124
torchsummary                          1.5.1
torchvision                           0.21.0+cu124
transformers                          4.51.3
